# Wandb for tracking

In [1]:
!pip install wandb -q

     |████████████████████████████████| 1.9 MB 10.6 MB/s 
     |████████████████████████████████| 182 kB 69.2 MB/s 
     |████████████████████████████████| 168 kB 78.0 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 168 kB 77.6 MB/s 
     |████████████████████████████████| 166 kB 79.2 MB/s 
     |████████████████████████████████| 166 kB 79.1 MB/s 
     |████████████████████████████████| 162 kB 85.7 MB/s 
     |████████████████████████████████| 162 kB 77.7 MB/s 
     |████████████████████████████████| 158 kB 61.4 MB/s 
     |████████████████████████████████| 157 kB 79.0 MB/s 
     |████████████████████████████████| 157 kB 85.5 MB/s 
     |████████████████████████████████| 157 kB 85.3 MB/s 
     |████████████████████████████████| 157 kB 86.3 MB/s 
     |████████████████████████████████| 157 kB 86.1 MB/s 
     |████████████████████████████████| 157 kB 80.9 MB/s 
     |████████████████████████████████| 157 kB 82.9 MB/s 
     |██████████

In [2]:
import wandb

In [3]:
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
%env WANDB_PROJECT=albert-base-ner

env: WANDB_PROJECT=albert-base-ner


# Data downloader


In [5]:
!pip install tokenizers -q
!pip install transformers -q
!pip install seqeval -q
!pip install evaluate -q
!pip install datasets -q

     |████████████████████████████████| 7.6 MB 14.4 MB/s 
     |████████████████████████████████| 5.8 MB 15.4 MB/s 
     |████████████████████████████████| 182 kB 82.0 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 72 kB 1.5 MB/s 
     |████████████████████████████████| 451 kB 26.0 MB/s 
     |████████████████████████████████| 212 kB 75.3 MB/s 
     |████████████████████████████████| 132 kB 65.7 MB/s 
     |████████████████████████████████| 127 kB 78.6 MB/s 


In [6]:
from datasets import load_dataset

dataset = load_dataset("conll2003")

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
labels_names = dataset["train"].features["ner_tags"].feature.names

In [8]:
labels_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [9]:
dataset["train"][:2]

{'id': ['0', '1'],
 'tokens': [['EU',
   'rejects',
   'German',
   'call',
   'to',
   'boycott',
   'British',
   'lamb',
   '.'],
  ['Peter', 'Blackburn']],
 'pos_tags': [[22, 42, 16, 21, 35, 37, 16, 21, 7], [22, 22]],
 'chunk_tags': [[11, 21, 11, 12, 21, 22, 11, 12, 0], [11, 12]],
 'ner_tags': [[3, 0, 7, 0, 0, 0, 7, 0, 0], [1, 2]]}

# Data preparing

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["tokens"], padding = "max_length", truncation = True, is_split_into_words = True)

In [12]:
tokenized_datasets_ = dataset.map(tokenize_function, batched = True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [13]:
len(tokenized_datasets_["train"][0]["input_ids"]) == len(tokenized_datasets_["train"][0]["ner_tags"])

False

Adjusting labels to fit input

In [14]:
def tokenize_adjust_labels(samples):
    tokenized_samples = tokenizer.batch_encode_plus(samples["tokens"], is_split_into_words = True, truncation = True)

    total_adjusted_labels = []

    for k in range(len(tokenized_samples["input_ids"])):
        prev_wid = -1

        word_ids_list = tokenized_samples.word_ids(batch_index = k)

        existing_label_ids = samples["ner_tags"][k]

        i = -1

        adjusted_label_ids = []

        for word_idx in word_ids_list:
            if word_idx is None:
                adjusted_label_ids.append(-100)
            elif word_idx != prev_wid:
                i += 1
                adjusted_label_ids.append(existing_label_ids[i])
                prev_wid = word_idx
            else:
                label_name = labels_names[existing_label_ids[i]]
                adjusted_label_ids.append(existing_label_ids[i])
        
        total_adjusted_labels.append(adjusted_label_ids)
            
    tokenized_samples["labels"] = total_adjusted_labels

    return tokenized_samples


In [15]:
out = tokenizer("Fine tune NER with BERT")
out

{'input_ids': [2, 1123, 6768, 13, 1031, 29, 11502, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
out.word_ids(0)

[None, 0, 1, 2, 2, 3, 4, None]

In [17]:
tokenized_dataset = dataset.map(tokenize_adjust_labels, batched = True, remove_columns = ["tokens", "ner_tags", "pos_tags", "chunk_tags"])

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [18]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [19]:
tokenized_dataset["train"][:2]

{'id': ['0', '1'],
 'input_ids': [[2, 2898, 12170, 18, 548, 645, 20, 16617, 388, 8624, 13, 9, 3],
  [2, 936, 14606, 3]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]],
 'labels': [[-100, 3, 0, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100], [-100, 1, 2, -100]]}

Padding token to a constant length

In [20]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

# Fine-tuning ALBERT for NER task

In [21]:
from transformers import AutoModelForTokenClassification

In [22]:
model = AutoModelForTokenClassification.from_pretrained("albert-base-v2", num_labels = len(labels_names))

Downloading:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForTokenClassification: ['predictions.decoder.bias', 'predictions.bias', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably

## Computing metrics 

In [23]:
import numpy as np
import evaluate

In [24]:
metric = evaluate.load("seqeval")

In [25]:
def compute_metrics(p):
    predictions, labels = p

    predictions = np.argmax(predictions, axis = 2)

    true_predictions = [
        [labels_names[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [labels_names[l] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions = true_predictions, references = true_labels)

    return {
        "precision" : results["overall_precision"],
        "recall" : results["overall_recall"],
        "f1" : results["overall_f1"],
        "accuracy" : results["overall_accuracy"]
    }

## Using Trainer API to fine-tune

In [26]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback, IntervalStrategy

In [27]:
batch_size = 16
logging_steps = len(tokenized_dataset["train"]) // batch_size
epochs = 7
weight_decay = .01
eval_steps = 100
learning_rate = 2e-5
early_stopping_patience = 3

In [28]:
training_args = TrainingArguments(
    output_dir="results",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    disable_tqdm=False,
    learning_rate = learning_rate,
    logging_steps=logging_steps,
    evaluation_strategy = IntervalStrategy.STEPS,
    eval_steps = eval_steps,
    save_total_limit = 5,
    metric_for_best_model = "eval_loss",
    load_best_model_at_end = True,
    report_to = "wandb"
) 

In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)]
)

In [30]:
trainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
100,No log,0.179526,0.798363,0.784809,0.791528,0.958537
200,No log,0.143634,0.804139,0.839599,0.821486,0.963336
300,No log,0.117137,0.864012,0.875537,0.869737,0.971118
400,No log,0.119293,0.820570,0.857568,0.838661,0.966446
500,No log,0.096108,0.883573,0.906074,0.894682,0.976485
600,No log,0.090703,0.896794,0.900452,0.898619,0.977706
700,No log,0.082952,0.891889,0.903098,0.897458,0.977422
800,No log,0.076507,0.891411,0.917650,0.904340,0.979027
900,0.139700,0.081648,0.906057,0.903759,0.904906,0.979311
1000,0.139700,0.071705,0.920564,0.913460,0.916999,0.981228


The following columns in the evaluation set don't have a corresponding argument in `AlbertForTokenClassification.forward` and have been ignored: id. If id are not expected by `AlbertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `AlbertForTokenClassification.forward` and have been ignored: id. If id are not expected by `AlbertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from results/checkpoint-2000 (score: 0.06482107192277908).


TrainOutput(global_step=2300, training_loss=0.08432804024737814, metrics={'train_runtime': 702.8154, 'train_samples_per_second': 139.848, 'train_steps_per_second': 8.745, 'total_flos': 83313359935476.0, 'train_loss': 0.08432804024737814, 'epoch': 2.62})

## Testing

In [31]:
predictions, labels, _ = trainer.predict(tokenized_dataset["test"])

predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)

true_predictions = [
    [labels_names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

true_labels = [
    [labels_names[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)

The following columns in the test set don't have a corresponding argument in `AlbertForTokenClassification.forward` and have been ignored: id. If id are not expected by `AlbertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3453
  Batch size = 16


In [32]:
results

{'LOC': {'precision': 0.8996265172735761,
  'recall': 0.9063969896519285,
  'f1': 0.9029990627928772,
  'number': 2126},
 'MISC': {'precision': 0.7270087124878993,
  'recall': 0.7263056092843327,
  'f1': 0.7266569908079342,
  'number': 1034},
 'ORG': {'precision': 0.8281811646297628,
  'recall': 0.8858131487889274,
  'f1': 0.8560282370425414,
  'number': 2601},
 'PER': {'precision': 0.9681170292573144,
  'recall': 0.9399126001456665,
  'f1': 0.9538063562453807,
  'number': 2746},
 'overall_precision': 0.8770729444508871,
 'overall_recall': 0.8890325614200071,
 'overall_f1': 0.8830122591943959,
 'overall_accuracy': 0.9744135136393982}

In [33]:
true_predictions

[['O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-PER', 'B-PER', 'I-PER', 'I-PER'],
 ['B-LOC',
  'B-LOC',
  'B-LOC',
  'O',
  'O',
  'B-LOC',
  'I-LOC',
  'I-LOC',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'I-MISC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O'],
 ['B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'O',
  'B-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'

In [34]:
true_labels

[['O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PER',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-PER', 'B-PER', 'I-PER', 'I-PER'],
 ['B-LOC',
  'B-LOC',
  'B-LOC',
  'O',
  'O',
  'B-LOC',
  'I-LOC',
  'I-LOC',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'I-MISC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O'],
 ['B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'O',
  'B-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'

In [35]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

In [36]:
y_pred = []
y = []
for sent in true_predictions:
    for tag in sent:
        y_pred.append(tag)
for sent in true_labels:
    for tag in sent:
        y.append(tag)


In [37]:
print(classification_report(y_pred, y))

              precision    recall  f1-score   support

       B-LOC       0.91      0.91      0.91      2134
      B-MISC       0.76      0.77      0.76      1023
       B-ORG       0.90      0.85      0.87      2754
       B-PER       0.94      0.97      0.96      2661
       I-LOC       0.80      0.88      0.84       251
      I-MISC       0.66      0.61      0.63       269
       I-ORG       0.87      0.81      0.84      1122
       I-PER       0.98      0.98      0.98      2440
           O       0.99      0.99      0.99     51755

    accuracy                           0.97     64409
   macro avg       0.87      0.86      0.87     64409
weighted avg       0.97      0.97      0.97     64409



In [38]:
print(f"Accuracy = {accuracy_score(y_pred, y):.6f}, \
        Precision (weighted) = {precision_score(y_pred, y, average = 'weighted'):.6f}, \
        Recall (weighted) = {recall_score(y_pred, y, average = 'weighted'):.6f}, \
        F1 (weighted) = {f1_score(y_pred, y, average = 'weighted'):.6f}")

Accuracy = 0.974414,         Precision (weighted) = 0.974137,         Recall (weighted) = 0.974414,         F1 (weighted) = 0.974211


In [39]:
from sklearn.metrics import confusion_matrix

In [40]:
print(confusion_matrix(y_pred, y))

[[ 1939    36    94    18     8     1     2     0    36]
 [   33   784    46    13     2    17     2     0   126]
 [   84   120  2339    90     0     6    10     0   105]
 [   16     9    27  2585     0     0     0     2    22]
 [    7     0     0     0   220     1    15     1     7]
 [    1    13     0     0     1   165     9     0    80]
 [    5     4    25     0    25    28   914    38    83]
 [    0     0     0    15     4     1    13  2403     4]
 [   41    68    70    25    14    32    91     2 51412]]


In [41]:
wandb.finish()

eval/accuracy,▁▂▄▃▆▆▆▆▇▇▆▇▇▇▇▇▇█████▇
eval/f1,▁▃▅▃▆▆▆▇▇▇▆▇▇▇▇█▇██████
eval/loss,█▆▄▄▃▃▂▂▂▂▂▂▁▁▁▂▂▁▁▁▁▁▁
eval/precision,▁▁▅▂▆▆▆▆▇█▆▇▇▇▆█▇██████
eval/recall,▁▄▅▄▇▆▆▇▆▇▆▇▇▇▇▇▇█████▇
eval/runtime,▂▂▁▁▂█▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▃▂
eval/samples_per_second,▆▇██▇▁█████▇█▇▇████▇█▅▇
eval/steps_per_second,▆▇██▇▁█████▇█▇▇████▇█▅▇
train/epoch,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇███
train/global_step,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇███
train/learning_rate,█▁
